Install and prepare package

In [ ]:
%ls

drive/  pywin32-306-cp311-cp311-win32.whl  sample_data/


In [ ]:
# !pip install pymupdf
# !pip install python-docx
# !pip install python-pptx
# !pip install pypandoc

ERROR: pywin32-306-cp311-cp311-win32.whl is not a supported wheel on this platform.


In [ ]:
import fitz, os, pandas as pd, numpy as np
import uuid, json, ast, traceback, csv
from docx import Document
from pptx import Presentation
import pypandoc

ModuleNotFoundError: No module named 'win32com'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Variable preparation

In [ ]:
base_dir = 'drive/MyDrive/project-ugm/'
share_drive = 'drive/Shareddrives/'

Read Link AES Folder

In [ ]:
def list_matakuliah(path):
  except_mk = ['Data.xlsx', 'Copy of Data.xlsx', 'Data Assignment.xlsx', 'Untitled document.gdoc']
  matakuliah = os.listdir(path)
  matakuliah = [matkul for matkul in matakuliah if matkul not in except_mk]

  return matakuliah


list_mk = list_matakuliah(os.path.join(share_drive, 'AES Dataset/Dataset'))
list_mk

['Agama Islam',
 'Bahasa Indonesia',
 'Bahasa Inggris',
 'Bahasa Inggris 1',
 'Belajar di Era Digital',
 'Ilmu Alamiah Dasar',
 'Ilmu Sosial & Budaya Dasar']

In [ ]:
def list_tugas(how_much):
  return [f'tugas {i+1}' for i in range(how_much)]

In [ ]:
def link_recursive(path):
  df = pd.DataFrame()

  directories_path = os.listdir(path)

  for root, directories, files in os.walk(path):
    root_file = root.split('/')

    if len(root_file) > 9 and len(files) != 0:
      df_loop = pd.DataFrame({'link': [f'{root}/{files[0]}']})
      df = pd.concat([df, df_loop], axis=0)


  return df


In [ ]:
df = link_recursive(os.path.join(share_drive, 'AES Dataset/Dataset'))
df

,link
0,drive/Shareddrives/AES Dataset/Dataset/Agama I...
0,drive/Shareddrives/AES Dataset/Dataset/Agama I...
0,drive/Shareddrives/AES Dataset/Dataset/Bahasa ...
0,drive/Shareddrives/AES Dataset/Dataset/Bahasa ...
0,drive/Shareddrives/AES Dataset/Dataset/Bahasa ...
...,...
0,drive/Shareddrives/AES Dataset/Dataset/Belajar...
0,drive/Shareddrives/AES Dataset/Dataset/Belajar...
0,drive/Shareddrives/AES Dataset/Dataset/Belajar...
0,drive/Shareddrives/AES Dataset/Dataset/Ilmu So...


In [ ]:
df.to_csv(os.path.join(base_dir, 'link_dataset_aes.csv'), index=False)

Converting PDF Table

In [ ]:
def penilaian_dosen(filename):
  lecture_assignment = pd.read_excel(filename)
  return lecture_assignment

In [ ]:
def open_pdf(name_file):
  return fitz.open(name_file)

In [ ]:
df_filename = pd.read_csv(os.path.join(base_dir, 'link_dataset_aes.csv'))
df_filename

,link
0,drive/Shareddrives/AES Dataset/Dataset/Agama I...
1,drive/Shareddrives/AES Dataset/Dataset/Agama I...
2,drive/Shareddrives/AES Dataset/Dataset/Bahasa ...
3,drive/Shareddrives/AES Dataset/Dataset/Bahasa ...
4,drive/Shareddrives/AES Dataset/Dataset/Bahasa ...
...,...
442,drive/Shareddrives/AES Dataset/Dataset/Belajar...
443,drive/Shareddrives/AES Dataset/Dataset/Belajar...
444,drive/Shareddrives/AES Dataset/Dataset/Belajar...
445,drive/Shareddrives/AES Dataset/Dataset/Ilmu So...


In [ ]:
def is_header_row(table):
  if len(table) == 0:
    return False

  first_row = table[0]

  if all(isinstance(cell, str) and cell.strip() for cell in first_row):
      return True
  return False

def down_read_table(table_extract):
  table_extract = [f'{[{table_extract[0][i]: row[i] for i in range(len(table_extract[0]))} for row in table_extract[1:]]}']
  return table_extract

def side_read_table(lists):
  lists = [[element for element in sublist if element not in [':', ';']] for sublist in lists]
  results = {key: value for key, value in lists}
  return [f'{[results]}']

def my_dict():
  my_df = {}

  for i in range(1, 10):
    my_df[f'json_{i}'] = '[{}]'

  return my_df

def convert_doc_to_docx(doc_path):
    docx_path = doc_path.replace('.doc', '.docx')
    pypandoc.convert_file(doc_path, 'docx', outputfile=docx_path)
    return docx_path

def add_new_value(array, new_value):
  keys = list(array[0].keys())

  if len(new_value[0]) != len(keys):
      raise ValueError("Jumlah nilai baru tidak sesuai dengan jumlah kunci")

  for value in new_value:
    new_object = dict(zip(keys, value))

    array.append(new_object)

  return array

def push_value_dict(table_extract, my_df, table_in):
  json_arrays = ast.literal_eval(my_df[f'json_{table_in}'][0])
  json_arrays = add_new_value(json_arrays, table_extract)

  return json_arrays

def check_direct(table_extract):
  return True if any((':' in element or ';' in element) for element in table_extract[0]) else False

def conditional_position_table(table_extract, is_header_row, check_direct,my_df):
  position = 'below'
  # cek jika tabel memiliki header dan baca kolom ke bawah
  if len(table_extract) > 0 and is_header_row(table_extract):
    # cek jika tabel tidak memiliki header dan baca kolom ke samping
    position = 'below'
    check_direct = check_direct(table_extract[0])
    if check_direct:
      position = 'side'

  # cek jika tabel tersambung ke halaman lain
  elif len(table_extract) > 0 and (table_extract[0][0].isdigit() or table_extract[0][0].lower() in ['no', 'id', '']):
    # cek jika tabel tidak memiliki header dan baca kolom ke samping
    position = 'next_page'
    check_direct = check_direct(table_extract[0])
    if check_direct:
      position = 'side'

  return position

def table_extracting_word(document):
  tables_data = []
  for table_index, table in enumerate(document.tables):
    table_data = []
    for row_index, row in enumerate(table.rows):
        row_data = [cell.text.strip() for cell in row.cells]
        table_data.append(row_data)

    tables_data.append(table_data)

  return tables_data

def table_extracting_ppt(slide):
  table_data = []
  table_id = 0
  for shape in slide.shapes:
    if shape.has_table:
      table_id += 1
      table = shape.table
      for row in table.rows:
        row_data = [cell.text for cell in row.cells]
        table_data.append(row_data)

  return table_data, table_id

In [ ]:
def read_pdf(filename):
  my_df = my_dict()
  jum_tables = 0
  table_in = 0

  pdf_doc = open_pdf(filename)

  df = pd.DataFrame()

  for i in range(len(pdf_doc)):
    page = pdf_doc[i]
    tabs = page.find_tables()

    is_table = True if len(tabs.tables) else False
    answer_type = f'pdf/{"table" if is_table else "text"}' if filename.endswith('.pdf') else f'text/{"table" if is_table else "text"}'

    for j, table in enumerate(tabs.tables):
      table_in += 1
      table_extract = np.array(table.extract())

      position = conditional_position_table(table_extract, is_header_row, check_direct, my_df)

      if position == 'below':
        my_df[f'json_{table_in}'] = down_read_table(table_extract)

      elif position == 'next_page':
        table_in -= 1
        json_arrays = push_value_dict(table_extract, my_df, table_in)
        my_df[f'json_{table_in}'] = [f'{json_arrays}']

      else:
        my_df[f'json_{table_in}'] = side_read_table(table_extract)

      my_df['answer_type'] = answer_type

  my_df['jumlah_tabel'] = table_in

  df_table = pd.DataFrame(my_df, columns=my_df.keys())
  df = pd.concat([df, df_table], axis=0, ignore_index=True)
  return df

def read_docx(filename):
  try:
        document = Document(filename)
  except Exception as e:
      print(f"Ada kesalahan file, mohon diperiksa: {e}")
      return []

  tables_data = []
  my_df = my_dict()
  df = pd.DataFrame()
  table_id = 0

  table_extracts = table_extracting_word(document)

  for table_extract in table_extracts:
    table_id += 1
    is_table = True if len(table_extract) else False
    answer_type = f'docx/{"table" if is_table else "text"}' if filename.endswith('.docx') else f'text/{"table" if is_table else "text"}'
    my_df['answer_type'] = answer_type

    position = conditional_position_table(table_extract, is_header_row, check_direct, my_df)

    if position == 'below':
      my_df[f'json_{table_id}'] = down_read_table(table_extract)

    elif position == 'next_page':
      table_id -= 1
      json_arrays = push_value_dict(table_extract, my_df, table_id)
      my_df[f'json_{table_id}'] = [f'{json_arrays}']

    else:
      my_df[f'json_{table_id}'] = side_read_table(table_extract)

  my_df['jumlah_tabel'] = table_id

  df_table = pd.DataFrame(my_df, columns=my_df.keys())
  df = pd.concat([df, df_table], axis=0, ignore_index=True)

  return df

def read_pptx(filename):
  try:
    prs = Presentation(filename)
  except Exception as e:
    print(f"Ada kesalahan file, mohon diperiksa: {e}")
    return []

  my_df = my_dict()
  df = pd.DataFrame()

  for slide in prs.slides:
    my_df['answer_type'] = 'pptx/table'

    table_extract, table_id = table_extracting_ppt(slide)

    if len(table_extract) == 0:
      continue

    position = conditional_position_table(table_extract, is_header_row, check_direct, my_df)

    if position == 'below':
      my_df[f'json_{table_id}'] = down_read_table(table_extract)

    elif position == 'next_page':
      table_id -= 1
      json_arrays = push_value_dict(table_extract, my_df, table_id)
      my_df[f'json_{table_id}'] = [f'{json_arrays}']

    else:
      my_df[f'json_{table_id}'] = side_read_table(table_extract)

  my_df['jumlah_tabel'] = table_id

  df_table = pd.DataFrame(my_df, columns=my_df.keys())
  df = pd.concat([df, df_table], axis=0, ignore_index=True)

  return df

In [ ]:
def write_problem_file(csv_path, data):
  with open(csv_path, 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

def take_mk_tugas(lists, strings):
  result = next((item for item in lists if item in strings), None)
  return result

def main(filename):
  try:
    print('process: ', filename)
    if filename.endswith('.pdf'):
      df = read_pdf(filename)
    elif filename.endswith('.docx') or filename.endswith('.doc'):
      if filename.endswith('.doc'):
        filename = convert_doc_to_docx(filename)

      df = read_docx(filename)
    elif filename.endswith('.pptx'):
      df = read_pptx(filename)
    else:
      write_problem_file(os.path.join(base_dir, 'problem_file.csv'), [[filename]])
      print(f'Sistem belum menentukan: {filename}')

    df['link'] = filename

    tasks = list_tugas(5)
    nama_mapel = take_mk_tugas(list_mk, filename)
    nama_tugas = take_mk_tugas(tasks, filename.lower())

    df['nama_tugas'] = nama_tugas
    df['nama_mapel'] = nama_mapel

    return df

  except Exception as e:
    print(e)
    pass

In [ ]:
new_df = read_pdf(os.path.join(share_drive, 'AES Dataset/JawabS2PPL/.pdf'))
new_df
# new_df.to_csv(os.path.join(base_dir, 'pptx.csv'))

[['' 'Aspek' '' '' 'Pendekatan Protipe' '' '' 'Pendekatan DevOps' '']
 ['Biaya' None None
  'Pada awal pengembangan\nakan lebih tinggi\nmemerlukan pembuatan\nprototipe yang berulang.\nBesaran biaya awal ini\ntergantung pada jumlah\niterasi yang dibutuhkan\nuntuk mencapai prototipe\nyang dibutuhkan.'
  None None
  'Biaya dapat lebih efisien\ndalam jangka panjang\nkarena fokus pada\notomatisasi, CI/CD, yang\nmengurangi biaya manual\ndan kesalahan'
  None None]]


KeyError: 'json_0'

In [ ]:
def index(filenames, name_saved='final_output.csv', is_saved=False):
  i = 0
  new_df = pd.DataFrame()

  for id in range(len(filenames)):
    filename = filenames[id]

    df = main(filename)

    if is_saved:
      try:
        df.to_csv(os.path.join(base_dir, name_saved), mode='a', index=False, header=False)
        print(f'Berhasil diproses: {i} - {filename}')
      except Exception as e:
        continue

    i += 1

    new_df = pd.concat([new_df, df], axis=0, ignore_index=True)

  return new_df


In [ ]:
index(df_filename['link'].values, 'final_output.csv', True)

In [ ]:
JawabS2PPL = os.listdir(os.path.join(share_drive, 'AES Dataset/JawabS2PPL'))
JawabS2PPL = [f'{os.path.join(share_drive, f"AES Dataset/JawabS2PPL/{ppl}")}' for ppl in JawabS2PPL if ppl.endswith('.pdf')]
JawabS2PPL

['drive/Shareddrives/AES Dataset/JawabS2PPL/Siti Fitriyati Ayu Kusuma Wardani-22508775PPA06445-PPLS2MMI.pdf',
 'drive/Shareddrives/AES Dataset/JawabS2PPL/Reka Alamsyah-22_509199_PPA_06452-PPLS2MMI.pdf',
 'drive/Shareddrives/AES Dataset/JawabS2PPL/Faridin-22_510916_PPA_06487-PPLS2MMI.pdf',
 'drive/Shareddrives/AES Dataset/JawabS2PPL/Copy of Reka Alamsyah-22_509199_PPA_06452-PPLS2MMI.pdf']

In [ ]:
index(JawabS2PPL, 'JawabS2PPL.csv', True)

process:  drive/Shareddrives/AES Dataset/JawabS2PPL/Siti Fitriyati Ayu Kusuma Wardani-22508775PPA06445-PPLS2MMI.pdf
If using all scalar values, you must pass an index
process:  drive/Shareddrives/AES Dataset/JawabS2PPL/Reka Alamsyah-22_509199_PPA_06452-PPLS2MMI.pdf
Berhasil diproses: 0 - drive/Shareddrives/AES Dataset/JawabS2PPL/Reka Alamsyah-22_509199_PPA_06452-PPLS2MMI.pdf
process:  drive/Shareddrives/AES Dataset/JawabS2PPL/Faridin-22_510916_PPA_06487-PPLS2MMI.pdf
'json_0'
process:  drive/Shareddrives/AES Dataset/JawabS2PPL/Copy of Reka Alamsyah-22_509199_PPA_06452-PPLS2MMI.pdf
Berhasil diproses: 1 - drive/Shareddrives/AES Dataset/JawabS2PPL/Copy of Reka Alamsyah-22_509199_PPA_06452-PPLS2MMI.pdf


,json_1,json_2,json_3,json_4,json_5,json_6,json_7,json_8,json_9,answer_type,json_10,json_11,jumlah_tabel,link,nama_tugas,nama_mapel
0,"[{'Aspek': 'Biaya', 'OO (Object Oriented)': 'P...","[{'Product Name': 'Product Owner', ':': ':', '...","[{'ID': '001', 'User Story': 'Sebagai pengguna...","[{'Sprint Code': 'Start Date', ':': ':', 'SPRI...","[{'ID': '1', 'User Story': 'Sebagai pengguna, ...","[{'Sprint Code': 'Start Date', ':': ':', 'SPRI...","[{'ID': '', 'User Story': 'Sebagai pengguna, s...","[{'Nama Produk': 'Logo', ':': ':', 'CaterEaseH...",[{'Domain Feature': 'Dashboard Penyedia\nLayan...,pdf/table,[{'Deskripsi Layanan': 'Hak dan Kewajiban Kedu...,[{'Kerahasiaan dan Perlindungan\nData': 'Perio...,11,drive/Shareddrives/AES Dataset/JawabS2PPL/Reka...,None,None
1,"[{'Aspek': 'Biaya', 'OO (Object Oriented)': 'P...","[{'Product Name': 'Product Owner', ':': ':', '...","[{'ID': '001', 'User Story': 'Sebagai pengguna...","[{'Sprint Code': 'Start Date', ':': ':', 'SPRI...","[{'ID': '1', 'User Story': 'Sebagai pengguna, ...","[{'Sprint Code': 'Start Date', ':': ':', 'SPRI...","[{'ID': '', 'User Story': 'Sebagai pengguna, s...","[{'Nama Produk': 'Logo', ':': ':', 'CaterEaseH...",[{'Domain Feature': 'Dashboard Penyedia\nLayan...,pdf/table,[{'Deskripsi Layanan': 'Hak dan Kewajiban Kedu...,[{'Kerahasiaan dan Perlindungan\nData': 'Perio...,11,drive/Shareddrives/AES Dataset/JawabS2PPL/Copy...,None,None


In [ ]:
df = pd.read_csv(os.path.join(base_dir, 'final_output.csv'))
df.head()

,json_1,json_2,json_3,json_4,json_5,json_6,json_7,json_8,json_9,answer_type,jumlah_tabel,link,nama_tugas,nama_mapel
0,[],[],[],[],[{}],[{}],[{}],[{}],[{}],pdf/table,4,drive/Shareddrives/AES Dataset/Dataset/Bahasa ...,tugas 1,Bahasa Indonesia
1,"[{'Indonesia': 'Pulau Jawa', 'm': None}]","[{'han ekonomi': 'Salamuddin Daeng', 'Indonesi...","[{'Daeng': 'onomi', 'juga me': 'Indonesia'}]","[{'mi': 'Indonesia', 'Indonesia': None, None: ...",[],[{}],[{}],[{}],[{}],pdf/table,5,drive/Shareddrives/AES Dataset/Dataset/Bahasa ...,tugas 3,Bahasa Indonesia
2,"[{'No': '1', 'Tugas Tutorial': 'Hari ini anda ...",[{}],[{}],[{}],[{}],[{}],[{}],[{}],[{}],docx/table,1,drive/Shareddrives/AES Dataset/Dataset/Bahasa ...,tugas 1,Bahasa Inggris
3,[],[],[{}],[{}],[{}],[{}],[{}],[{}],[{}],docx/table,2,drive/Shareddrives/AES Dataset/Dataset/Bahasa ...,tugas 1,Bahasa Inggris
4,"[{'ISHAK HINDUN': '', None: None}, {'ISHAK HIN...",[{}],[{}],[{}],[{}],[{}],[{}],[{}],[{}],pdf/table,1,drive/Shareddrives/AES Dataset/Dataset/Bahasa ...,tugas 1,Bahasa Inggris
